# Iowa house price prediction
### using DecisionTreeRegressor
#### By Comparing Different Tree Sizes and chooing the right size

In [1]:
# Code you have previously used to load data
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

import datetime 

In [2]:
# Path of the file to read
subdirectory = "data"
filename = "forex_AUDCAD.csv"

# Construct the full file path
file_path = os.path.join(subdirectory, filename)

<IPython.core.display.Javascript object>

In [3]:
forex_data = pd.read_csv(file_path)
forex_data.describe()

,topen,theigh,tlow,tclose,tvolume
count,5.737313e+06,5.737313e+06,5.737313e+06,5.737313e+06,5737313.0
mean,9.575396e-01,9.576482e-01,9.574305e-01,9.575389e-01,0.0
std,5.367333e-02,5.364885e-02,5.369731e-02,5.367285e-02,0.0
min,7.187000e-01,7.226000e-01,7.172000e-01,7.196000e-01,0.0
25%,9.217800e-01,9.218700e-01,9.216800e-01,9.217800e-01,0.0
50%,9.587800e-01,9.588700e-01,9.586800e-01,9.587800e-01,0.0
75%,9.962400e-01,9.963400e-01,9.961300e-01,9.962400e-01,0.0
max,1.078080e+00,1.078190e+00,1.078020e+00,1.078110e+00,0.0


In [4]:
# Convert the 'time_column' to minutes
def convert_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    total_minutes = hours * 60 + minutes
    return total_minutes

In [5]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [6]:
# Feature Engineering part
## Create a new column with value number of days since the minimum days of the column
## to make the use a dare column as a numeric value. Since Decision Tree gaven an error date columns.

forex_data['tdate'] = pd.to_datetime(forex_data['tdate'])
min_date = forex_data['tdate'].min()
forex_data['nDate'] = (forex_data['tdate'] - min_date).dt.days +1 # +1 is because be dont want zero as a date difference

forex_data['nMinutes'] = forex_data['ttime'].apply(convert_to_minutes)

forex_data.head()

,tick,tdate,ttime,topen,theigh,tlow,tclose,tvolume,nDate,nMinutes
0,AUDCAD,2007-10-07,05:02,0.8822,0.8823,0.8822,0.8822,0,1,302
1,AUDCAD,2007-10-07,05:01,0.8822,0.8822,0.8822,0.8822,0,1,301
2,AUDCAD,2007-10-07,05:00,0.8823,0.8823,0.8823,0.8823,0,1,300
3,AUDCAD,2007-10-14,05:00,0.8799,0.8799,0.8799,0.8799,0,8,300
4,AUDCAD,2007-10-21,05:02,0.8595,0.8595,0.8594,0.8594,0,15,302


In [7]:
#  >> 1. Split data in to features and target -------------------
y = forex_data.tclose

features = ['nDate','nMinutes','topen','theigh','tlow']
X = forex_data[features]
#  <<  1. Split data in to features and target -------------------


#  >>  2. Split into validation and training data-----------------
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
#  <<  2. Split into validation and training data-----------------

In [8]:
#  >> 3. Create function to retuen Mean Absolute Error (MAE) for givem number of leaves -------------------
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)
#  << 3. Create function to retuen Mean Absolute Error (MAE) for givem number of leaves -------------------

In [9]:
#  >> 4. Create list with number of leaves notes for testing -------------------
candidate_max_leaf_nodes =  [x for x in range(2, 200)]
#print(candidate_max_leaf_nodes)
#  << 4. Create list with number of leaves notes for testing -------------------

In [ ]:
#  >> 5. Find best leaves size -------------------
scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}
best_tree_size = min(scores, key=scores.get)

#  >> 5. Find best leaves size -------------------

In [ ]:
#  >> 6. Specify and fit the mode  --------------------------------------
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)

# fit the final model and uncomment the next two lines
final_model.fit(X, y)
#  << 6. Specify and fit the mode  --------------------------------------

In [ ]:
#  >> 7. Predict ------------------------------------------------
val_predictions = final_model.predict(val_X)
#  >> 7. Predict ------------------------------------------------

#  >> 8. calculate mean absolute error --------------------------
val_mae = mean_absolute_error(val_predictions, val_y)
#  << 8. calculate mean absolute error --------------------------

print("Validation MAE: {:,.0f}".format(val_mae))


In [ ]:
print(best_tree_size)